## Retrain the YOLO model for your own dataset.

### Setup

Remember: 

* select GPU used for model training and
* run jupyter notebook on the port that is not used e.g.

$    jupyter notebook --port 5555

In [ ]:
# select GPU used for model training
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [1]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from yolo3.model import preprocess_true_boxes, yolo_body, tiny_yolo_body, yolo_loss
from yolo3.utils import get_random_data

from keras_utils import *

Using TensorFlow backend.


In [2]:
annotation_path = 'Hardhat_trainval.txt'
log_dir = 'logs/000/'
classes_path = 'model_data/GDUT-HWD_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416,416) # multiple of 32, hw

In [3]:
is_tiny_version = len(anchors)==6 # default setting
if is_tiny_version:
    model = create_tiny_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/tiny_yolo_weights.h5')
else:
    model = create_model(input_shape, anchors, num_classes,
        freeze_body=2, weights_path='model_data/yolo.h5') # make sure you know what you freeze

logging = TensorBoard(log_dir=log_dir)
checkpoint = ModelCheckpoint(log_dir + 'ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5',
    monitor='val_loss', save_weights_only=True, save_best_only=True, period=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)

Create YOLOv3 model with 9 anchors and 6 classes.


/home/krittametht/.local/lib/python3.6/site-packages/keras/engine/topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 33) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
/home/krittametht/.local/lib/python3.6/site-packages/keras/engine/topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((33,) vs (255,)).
  weight_values[i].shape))
/home/krittametht/.local/lib/python3.6/site-packages/keras/engine/topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 33) vs (255, 512, 1, 1)).
  weight_values[i].shape))
/home/krittametht/.local/lib/python3.6/site-packages/keras/engine/topology.py:3473: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((33,) vs (255,)).
  weight_values[i].shape))
/home/krittametht/.local/lib/python3.6/site-packages/keras/engine/topology.py:3473: UserWarnin

Load weights model_data/yolo.h5.
Freeze the first 249 layers of total 252 layers.


In [4]:
val_split = 0.1
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

### Stage 1: Train with frozen layers to get a stable loss

* Adjust num epochs to your dataset. 
* This step is enough to obtain a not bad model.

In [5]:
# > Adjust batch_size and num_epochs

batch_size = 32

num_epochs = 50

In [6]:
# Train with frozen layers first, to get a stable loss.
# Adjust num epochs to your dataset. This step is enough to obtain a not bad model.
if True:
    model.compile(optimizer=Adam(lr=1e-3), loss={
        # use custom yolo_loss Lambda layer.
        'yolo_loss': lambda y_true, y_pred: y_pred})

    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
            steps_per_epoch=max(1, num_train//batch_size),
            validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
            validation_steps=max(1, num_val//batch_size),
            epochs=num_epochs,
            initial_epoch=0,
            callbacks=[logging, checkpoint])
    model.save_weights(log_dir + 'trained_weights_stage_1.h5')

Train on 1429 samples, val on 158 samples, with batch size 4.
Epoch 1/50
  4/357 [..............................] - ETA: 23:23 - loss: 8968.7887

KeyboardInterrupt: 

### Stage 2: Unfreeze and continue training, to fine-tune.
* Train longer if the result is not good.

*** To do just stage 2
1. Run all cells in 'Setup' section.
2. Uncomment and run the following cell.
3. Run the last cell for continuing model training.

In [ ]:
#model.load_weights(log_dir + 'trained_weights_stage_1.h5')

In [ ]:
# > Adjust batch_size_2 and num_epochs_2

batch_size_2 = 32

num_epochs_2 = 100

In [ ]:
if True:
    for i in range(len(model.layers)):
        model.layers[i].trainable = True
    model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
    print('Unfreeze all of the layers.')

    # batch_size_2 = 32 # note that more GPU memory is required after unfreezing the body
    print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size_2))
    model.fit_generator(data_generator_wrapper(lines[:num_train], batch_size_2, input_shape, anchors, num_classes),
        steps_per_epoch=max(1, num_train//batch_size_2),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size_2, input_shape, anchors, num_classes),
        validation_steps=max(1, num_val//batch_size_2),
        epochs=num_epochs_2,
        initial_epoch=50,
        callbacks=[logging, checkpoint, reduce_lr, early_stopping])
    model.save_weights(log_dir + 'trained_weights_final.h5')

# Further training if needed.